In [1]:
import oracledb
import altair as alt
import numpy as np
import pandas as pd
import re
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder
import statsmodels.api as sm
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import shap

/opt/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
dsn = oracledb.makedsn("localhost", 1522, service_name="stu")
connection = oracledb.connect(user="ora_dty200", password="a65544413", dsn=dsn)

cursor = connection.cursor()

# EDA

Firstly, we want to start with querying data for some exploratory data analysis to examine the overall trend for each variable. Due to the scope of this project assignment, we may not be able to discuss in detail the trend in every plot, but we will include some major findings by the end of this part.

In [3]:
# Query to retrieve percentage lost data for each year for the whole USA
query = """
    SELECT Year, AVG(PercentLost) AS AveragePercentLost
    FROM Bee
    GROUP BY Year
    ORDER BY Year DESC
"""

# Execute the query
cursor.execute(query)

# Fetch all rows
rows = cursor.fetchall()

# Extract data into separate lists for plotting
years = []
percent_lost = []

for row in rows:
    years.append(row[0])
    percent_lost.append(row[1])

# Make a dataframe for the plot
data = {"years": years,
        "percent_lost": percent_lost}

data = pd.DataFrame(data)

# Create a line graph
alt.Chart(data).mark_line(color = "skyblue", point = True).encode(
    x = alt.X("years:O", title = 'Year', axis=alt.Axis(labelAngle= 0)),
    y = alt.Y("percent_lost:Q", title = 'Average Percentage Lost (%)', scale = alt.Scale(domain = [13, 15.6])),
    tooltip = [alt.Tooltip("years", title = "Year"),
               alt.Tooltip("percent_lost", title = "Average Percentage Lost (%)", format='.2f')]
).properties(
    title = 'Average Percentage Lost of Bee Colonies in the USA Over Time',
    width = 600,
    height = 300
).configure_axis(
    grid = True
)

alt.Chart(...)

From the EDA of the whole USA data, we can observe that the average percentage loss of bee colonies differs over the 5-year period. The average percentage loss of bee colonies slightly increased from 2015 to 2016, then decreased from 15.5% to about 13.1% from 2016 to 2017. However, this percentage loss continued to increase for 2018 and 2019. The highest percentage loss occurred in 2016 (15.5%).

In [4]:
query_temperature = """
    SELECT Year, AVG(AverageTemperature) AS AverageTemperature
    FROM MonitorStation
    GROUP BY Year
    ORDER BY Year DESC
"""

# Execute the temperature query
cursor.execute(query_temperature)

# Fetch all temperature rows
temperature_rows = cursor.fetchall()

# Extract data into separate lists for plotting
years_temperature = []
average_temperature = []
for row in temperature_rows:
    years_temperature.append(row[0])
    average_temperature.append(row[1])

# Make a dataframe for the plot
data = {"years_temperature": years_temperature,
        "average_temperature": average_temperature}

data = pd.DataFrame(data)

# Create a line graph
alt.Chart(data).mark_line(color = "orange", point = {"color":"orange", "size": 60}).encode(
    x = alt.X("years_temperature:O", title = 'Year', axis=alt.Axis(labelAngle= 0)),
    y = alt.Y("average_temperature:Q", title = 'Average Temperature (°C)', scale = alt.Scale(domain = [53, 56.5])),
    tooltip = [alt.Tooltip("years_temperature", title = "Year"),
               alt.Tooltip("average_temperature", title = "Average Temperature (°C)", format='.2f')]
).properties(
    title = 'Average Temperature in the USA Over Time',
    width = 600,
    height = 300
).configure_axis(
    grid = True
)

alt.Chart(...)

Regarding average temperature, it increased from 54.8°C to 56.1°C from 2015 to 2016, reaching the highest temperature among the 5 years, then decreased over the following three years. Since 2016 had both the highest average temperature and the highest average percentage loss for the USA, it may imply a potential association between temperature and bee health.

In [5]:
# Query to retrieve average pesticide usage data for each year for the whole USA
query = """
    SELECT Year, AVG((LowEstimate + HighEstimate) / 2) AS AveragePesticideUsage
    FROM Pesticide
    GROUP BY Year
    ORDER BY Year
"""

# Execute the query
cursor.execute(query)

# Fetch all rows
rows = cursor.fetchall()

# Extract data into separate lists for plotting
years = []
average_pesticide_usage = []

for row in rows:
    years.append(row[0])
    average_pesticide_usage.append(row[1])


# Make a dataframe for the plot
data = {"years": years,
        "average_pesticide_usage": average_pesticide_usage}

data = pd.DataFrame(data)

# Create a line graph
alt.Chart(data).mark_line(color = "green", point = {"color":"green", "size": 60}).encode(
    x = alt.X("years:O", title = 'Year', axis=alt.Axis(labelAngle= 0)),
    y = alt.Y("average_pesticide_usage:Q", title = 'Average Pesticide Usage (kg)', scale = alt.Scale(domain = [5200, 5900])),
    tooltip = [alt.Tooltip("years", title = "Year"),
               alt.Tooltip("average_pesticide_usage", title = "Average Pesticide Usage (kg)", format='.2f')]
).properties(
    title = 'Average Pesticide Usage in the USA Over Time',
    width = 600,
    height = 300
).configure_axis(
    grid = True
)

alt.Chart(...)

In terms of pesticide usage in the USA, the amount increased from 5300kg in 2015 to its peak usage, approximately 5890kg in 2016. Subsequently, the usage decreased over time until 2018, reaching about 5310kg. The usage then increased to approximately 5750kg in 2019. Since 2016 had both the highest pesticide usage and the highest average percentage loss of bees for the USA, it may imply a potential association between pesticide usage and bee health.

In [6]:
# Query to retrieve average AQI data of gas conditions for each gas for each year for the whole USA
query = """
    SELECT Year, Name, AVG(AverageAQI) AS AverageAQI
    FROM GasConditions
    GROUP BY Year, Name
    ORDER BY Year, Name
"""

# Execute the query
cursor.execute(query)

# Fetch all rows
rows = cursor.fetchall()

# Extract data into separate lists for plotting
years = []
gases = []
average_aqi_values = []

for row in rows:
    years.append(row[0])
    gases.append(row[1])
    average_aqi_values.append(row[2])

# Get unique gases
unique_gases = list(set(gases))

# Make a dataframe for the plot
data = {"years": years,
        "gases": gases, 
        "average_aqi_values": average_aqi_values}

data = pd.DataFrame(data)

# Create a line graph
alt.Chart(data).mark_line(point = True).encode(
    x = alt.X("years:O", title = 'Year', axis=alt.Axis(labelAngle= 0)),
    y = alt.Y("average_aqi_values:Q", title = 'Average AQI'),
    color = "gases",
    tooltip = [alt.Tooltip("years", title = "Year"),
               alt.Tooltip("gases", title = "Gase Type"),
               alt.Tooltip("average_aqi_values", title = "Average AQI", format='.2f')]
).properties(
    title = 'Average AQI of Gas Conditions in the USA Over Time',
    width = 600,
    height = 300
).configure_axis(
    grid = True
)

alt.Chart(...)

In terms of Average AQI, all four gases do not seem to change much between years in the USA. Since a higher AQI value indicates greater levels of air pollution and greater health concerns, O3 has the highest value of the Average AQI, posing the highest risk of contributing to bee health among these four gases, followed by NO2, CO, and SO2. 

Next, we select the top 10 states with the highest percentage loss of bees to conduct a closer examination of the relationship between different factors and bee health.

## Top 10 states with the most percentage lost of Bees

In [7]:
# Query to retrieve top 10 states with the most percentage lost of bees
query = """
    SELECT State, AVG(PercentLost) AS AveragePercentLost
    FROM Bee
    GROUP BY State
    ORDER BY AveragePercentLost DESC
    FETCH FIRST 10 ROWS ONLY
"""

# Execute the query
cursor.execute(query)

# Fetch all rows
rows = cursor.fetchall()

# Extract data into separate lists for plotting
states = []
percent_lost = []

for row in rows:
    states.append(row[0])
    percent_lost.append(row[1])

# Make a dataframe for the plot
data = {"states": states,
        "percent_lost": percent_lost}

data = pd.DataFrame(data)

# Plot data using a bar chart
alt.Chart(data).mark_bar(color = "skyblue").encode(
    x = alt.X("states:N", title = 'State', axis=alt.Axis(labelAngle= -45)),
    y = alt.Y("percent_lost:Q", title = 'Average Percentage Lost (%)'),
    tooltip = [alt.Tooltip("states", title = "State"),
               alt.Tooltip("percent_lost", title = "Average Percentage Lost (%)", format='.2f')]
).properties(
    title = 'Top 10 States with the Most Percentage Lost of Bees',
    width = 600,
    height = 300
)

alt.Chart(...)

According to the bar chart above, we can see that the average percentage loss of bees for these top 10 states ranges from about 17.2% to 21%.

In [8]:
# Query to retrieve the top 10 states with the most percentage lost of bees
query = """
    SELECT State, AVG(PercentLost) AS AveragePercentLost
    FROM Bee
    GROUP BY State
    ORDER BY AveragePercentLost DESC
    FETCH FIRST 10 ROWS ONLY
"""

# Execute the query
cursor.execute(query)

# Fetch all rows
rows = cursor.fetchall()

# Extract state names from the fetched rows
top_10_states = [row[0] for row in rows]

# Print the names of the top 10 states
print("Top 10 States with the Most Percentage Lost:")
for state in top_10_states:
    print(state)

Top 10 States with the Most Percentage Lost:
Utah
Washington
Illinois
Massachusetts
Virginia
North Dakota
New Jersey
Kansas
Georgia
Texas


In [ ]:
# Top 10 States with the Most Percentage Lost
states = ['Utah', 'Washington', 'Illinois', 'Massachusetts', 'Virginia', 'North Dakota', 'New Jersey', 'Kansas', 'Georgia', 'Texas']

for state in states:
    # Fetch data for percentage lost by disease
    def fetch_percentage_diseaselost_data():
        query = f"""
            SELECT Year, PercentLostByDisease
            FROM Bee
            WHERE State = '{state}'
            ORDER BY Year
        """
        cursor.execute(query)
        rows = cursor.fetchall()
        return rows

    # Fetch percentage affected by parasites data for the state from the Parasite table
    def fetch_parasite_data():
        query = f"""
            SELECT Year, PercentAffected
            FROM Parasite
            WHERE State = '{state}'
            ORDER BY Year
        """
        cursor.execute(query)
        rows = cursor.fetchall()
        return rows

    # Fetch colony tracker data
    def fetch_colony_tracker_data():
        query = f"""
            SELECT Year, Colony, LostColony, AddColony
            FROM Bee
            WHERE State = '{state}'
            ORDER BY Year
        """
        cursor.execute(query)
        rows = cursor.fetchall()
        return rows

    # Fetch pesticide usage data for the state
    def fetch_pesticide_usage_data():
        query = f"""
            SELECT Year, LowEstimate, HighEstimate
            FROM Pesticide
            WHERE State = '{state}'
            ORDER BY Year
        """
        cursor.execute(query)
        rows = cursor.fetchall()
        return rows

    # Fetch AQI data for each gas in the state
    def fetch_aqi_data():
        query = f"""
            SELECT Year, Name, AverageAQI
            FROM GasConditions
            WHERE State = '{state}'
            ORDER BY Year
        """
        cursor.execute(query)
        rows = cursor.fetchall()
        return rows

    # Fetch temperature data for the state
    def fetch_temperature_data():
        query = f"""
        SELECT m.Year, m.AverageTemperature
        FROM MonitorStation m
        JOIN Detect d ON m.CentroidLongitude = d.CentroidLongitude 
                     AND m.CentroidLatitude = d.CentroidLatitude
                     AND m.Year = d.StationYear
        WHERE d.BeeState = '{state}'
        ORDER BY m.Year
        """
        cursor.execute(query)
        rows = cursor.fetchall()
        return rows

    # Fetch data for percentage lost by disease
    percentage_diseaselost_data = fetch_percentage_diseaselost_data()
    years_lost = [row[0] for row in percentage_diseaselost_data]
    percentage_lost_by_disease = [row[1] for row in percentage_diseaselost_data]

    # Fetch data for percentage affected by parasites
    parasite_data = fetch_parasite_data()
    years_parasite = [row[0] for row in parasite_data]
    percentage_parasite = [row[1] for row in parasite_data]

    # Fetch data for colony tracker
    colony_tracker_data = fetch_colony_tracker_data()
    colony_years = [row[0] for row in colony_tracker_data]
    colony_values = [row[1] for row in colony_tracker_data]
    lost_colonies = [row[2] for row in colony_tracker_data]
    added_colonies = [row[3] for row in colony_tracker_data]

    # Fetch pesticide usage data
    pesticide_data = fetch_pesticide_usage_data()
    years = [row[0] for row in pesticide_data]
    low_estimate = [row[1] for row in pesticide_data]
    high_estimate = [row[2] for row in pesticide_data]

    # Fetch AQI data for each gas
    aqi_data = fetch_aqi_data()

    # Separate AQI data for each gas
    gas_names = set(row[1] for row in aqi_data)
    gas_data = {gas_name: ([], []) for gas_name in gas_names}

    for row in aqi_data:
        year, gas_name, aqi_value = row
        gas_data[gas_name][0].append(year)
        gas_data[gas_name][1].append(aqi_value)

    # Fetch temperature data for the state
    temperature_data = fetch_temperature_data()
    years = [row[0] for row in temperature_data]
    temperatures = [row[1] for row in temperature_data]

    # Plot 1: Percentage lost by disease and parasite
    plot_1_data = pd.DataFrame({"year": years_parasite + years_lost,
                                "percentage" : percentage_parasite + percentage_lost_by_disease,
                                "type": ["Percentage Affected by Parasites"] * len(years_parasite) + ["Percentage Lost by Disease"] * len(years_lost)})


    globals()[re.sub(r'[^a-zA-Z0-9]', '_', str(state)) + "_plot_1"] = alt.Chart(plot_1_data).mark_line(point = True).encode(
        x = alt.X("year:O", title = "Year", axis=alt.Axis(labelAngle=0)), 
        y = alt.Y("percentage:Q", title = "Percentage Lost"),
        color = alt.Color("type:N", scale = alt.Scale(range = ["red", "blue"]), legend=alt.Legend(title='Legend')),
        tooltip = [alt.Tooltip("year", title = "Year"),
                   alt.Tooltip("percentage", title = "Percentage Lost", format='.2f')]
    ).properties(
        title = (f'Percentage Lost by Disease and Parasite in {state}'),
        width = 500,
        height = 300
    )

    # Plot 2: Colony tracker
    plot_2_data = pd.DataFrame({"year": colony_years + colony_years + colony_years,
                                "number": colony_values + lost_colonies + added_colonies,
                                "type": ["Total Colonies"] * len(colony_years) +  ["Lost Colonies"] * len(colony_years) + ["Added Colonies"] * len(colony_years)})

    globals()[re.sub(r'[^a-zA-Z0-9]', '_', str(state)) + "_plot_2"] = alt.Chart(plot_2_data).mark_line(point = True).encode(
        x = alt.X("year:O", title = "Year", axis=alt.Axis(labelAngle=0)), 
        y = alt.Y("number:Q", title = 'Number of Colonies'),
        color = alt.Color("type:N", scale = alt.Scale(domain = ["Total Colonies", "Lost Colonies", "Added Colonies"],
                                                      range = ["black", "red", "green"])),
        tooltip = [alt.Tooltip("year", title = "Year"),
                   alt.Tooltip("number", title = "Number of Colonies")]
    ).properties(
        title = (f'Colony Tracker in {state}'),
        width = 500,
        height = 300
    )
    
    # Plot 3: Pesticide usage 
    plot_3_data = pd.DataFrame({"year": years + years,
                                "number": low_estimate + high_estimate,
                                "type": ["Low Estimate"] * len(years) +  ["High Estimate"] * len(years)})

    globals()[re.sub(r'[^a-zA-Z0-9]', '_', str(state)) + "_plot_3"] = alt.Chart(plot_3_data).mark_line(point = True).encode(
        x = alt.X("year:O", title = "Year", axis=alt.Axis(labelAngle=0)), 
        y = alt.Y("number:Q", title = 'Pesticide Usage (kg)'),
        color = alt.Color("type:N", scale = alt.Scale(domain = ["Low Estimate","High Estimate"],
                                                      range = ["blue", "green"])),
        tooltip = [alt.Tooltip("year", title = "Year"),
                   alt.Tooltip("number", title = "Pesticide Usage (kg)")]
    ).properties(
        title = (f'Pesticide Usage in {state} Over the Years'),
        width = 500,
        height = 300
    )

    # Plot 4: Average AQI for each gas
    plot_4_data = pd.DataFrame(aqi_data, columns = ["year", "type", "value"])

    globals()[re.sub(r'[^a-zA-Z0-9]', '_', str(state)) + "_plot_4"] = alt.Chart(plot_4_data).mark_line(point = True).encode(
        x = alt.X("year:O", title = "Year", axis=alt.Axis(labelAngle=0)), 
        y = alt.Y("value:Q", title = 'Average AQI'),
        color = alt.Color("type:N", scale = alt.Scale(domain = ["CO", "O3", "NO2", "SO2"],
                                                      range = ["blue", "orange", "green", "red"])),
        tooltip = [alt.Tooltip("year", title = "Year"),
                   alt.Tooltip("value", title = "Average AQI", format='.2f')]
    ).properties(
        title = (f'AQI for Each Gas in {state} Over the Years'),
        width = 500,
        height = 300
    )
   
    # Plot 5: Average temperature change 
    plot_5_data = pd.DataFrame({"year": years,
                                "temperatures": temperatures})

    globals()[re.sub(r'[^a-zA-Z0-9]', '_', str(state)) + "_plot_5"] = alt.Chart(plot_5_data).mark_line(point = True).encode(
        x = alt.X("year:O", title = "Year", axis=alt.Axis(labelAngle=0)), 
        y = alt.Y("temperatures:Q", title = 'Average Temperature (°C)', scale = alt.Scale(zero=False)),
        tooltip = [alt.Tooltip("year", title = "Year"),
                   alt.Tooltip("temperatures", title = "Average Temperature (°C)", format='.2f')]
    ).properties(
        title = (f'Temperature Change in {state} Over the Years'),
        width = 500,
        height = 300
    )

In [ ]:
# Vertically and horizontally concatenate the change in percentage of lost by disease and parasite over years of all states into a single plot called plot_1

v1_1 = alt.hconcat(Utah_plot_1, Washington_plot_1, Illinois_plot_1, Massachusetts_plot_1, Virginia_plot_1)
v1_2 = alt.hconcat(North_Dakota_plot_1, New_Jersey_plot_1, Kansas_plot_1, Georgia_plot_1, Texas_plot_1)
plot_1 = alt.vconcat(v1_1, v1_2)
plot_1

In general, parasites appear to have the most significant impact on bee colonies compared to diseases among these states. Additionally, in most of the states, the percentage loss of bees due to parasites shows an increasing trend over the period between 2015 and 2019.

In [ ]:
# Vertically and horizontally concatenate the change in colonies over years of all states into a single plot called plot_2
v2_1 = alt.hconcat(Utah_plot_2, Washington_plot_2, Illinois_plot_2, Massachusetts_plot_2, Virginia_plot_2)
v2_2 = alt.hconcat(North_Dakota_plot_2, New_Jersey_plot_2, Kansas_plot_2, Georgia_plot_2, Texas_plot_2)
plot_2 = alt.vconcat(v2_1, v2_2)
plot_2

Among these 10 states, all of them show decreasing trends for the total number of colonies except for Massachusetts and North Dakota. The number of added colonies and the number of lost colonies fluctuate but remain generally stable.

In [ ]:
# Vertically and horizontally concatenate the change in pesticide usage over years of all states into a single plot called plot_3

v3_1 = alt.hconcat(Utah_plot_3, Washington_plot_3, Illinois_plot_3, Massachusetts_plot_3, Virginia_plot_3)
v3_2 = alt.hconcat(North_Dakota_plot_3, New_Jersey_plot_3, Kansas_plot_3, Georgia_plot_3, Texas_plot_3)
plot_3 = alt.vconcat(v3_1, v3_2)
plot_3

In terms of pesticide usage, all 10 states fluctuate and show an overall increasing trend in pesticide usage except for Kansas and Virginia, which show an overall decreasing trend. Texas showed an increase from 2015 to 2018 and a decreasing trend since 2019. Massachusetts showed a drastic increase in pesticide usage from 2017 to 2018.

In [ ]:
# Vertically and horizontally concatenate the change in AQI for each gas over years of all states into a single plot called plot_4

v4_1 = alt.hconcat(Utah_plot_4, Washington_plot_4, Illinois_plot_4, Massachusetts_plot_4, Virginia_plot_4)
v4_2 = alt.hconcat(North_Dakota_plot_4, New_Jersey_plot_4, Kansas_plot_4, Georgia_plot_4, Texas_plot_4)
plot_4 = alt.vconcat(v4_1, v4_2)
plot_4

The average AQI of O3 has the highest value, followed by NO2, CO, and SO2 among all the states. The average AQI of O3 and NO2 fluctuates a lot in Utah from 2015 to 2019, while they are quite stable for all the other states.

In [ ]:
# Vertically and horizontally concatenate the temperature change over years of all states into a single plot called plot_5

v5_1 = alt.hconcat(Utah_plot_5, Washington_plot_5, Illinois_plot_5, Massachusetts_plot_5, Virginia_plot_5)
v5_2 = alt.hconcat(North_Dakota_plot_5, New_Jersey_plot_5, Kansas_plot_5, Georgia_plot_5, Texas_plot_5)
plot_5 = alt.vconcat(v5_1, v5_2)

plot_5

The temperature fluctuates for all states by about 1-2°C between years, except for North Dakota and Kansas, which showed a steady decrease of 2°C per year for North Dakota and 1°C per year for Kansas.

### Preprocessing

Now we want to fetch all the necessary tables with factors that have an influential impact on bee colony losses, namely the Name of Gas Pollutant and their yearly average concentration in ppm, temperature change, percentage colony loss by disease, and pesticide usage, where the percentage loss of bee colonies are the response variable. Also for pesticide usage, since the low estimate amount and high estimate are pretty similar, we want to use their average. In addition, in terms of gas pollutants, from EDA we observe there are only 1 unique value for O3, which is clearly not useful for regression analysis, we will exclude them in further analysis.

In [ ]:
#create the dataframe for GasConditions
query = """
    SELECT *
    FROM GasConditions 
"""

# Execute the query
cursor.execute(query)

# Fetch all rows
rows = cursor.fetchall()

columns = ['GasName', 'State', 'Year', 'MeanValue', 'AverageAQI']

# Create a DataFrame
gas_conditions_df = pd.DataFrame(rows, columns=columns)

print(gas_conditions_df)

In [ ]:
# SQL query to select all columns from the Bee table
query_bee = """
    SELECT *
    FROM Bee
"""
# Execute the SQL query to retrieve data from the Bee table
cursor.execute(query_bee)

# Fetch all rows from the executed SQL query
rows_bee = cursor.fetchall()

# Column headers as they appear in the Bee table of the database
# The order of these column names should exactly match the order of columns in the database table
columns_bee = ['State', 'Year', 'MaxColony', 'LostColony', 'PercentLost', 
               'Colony', 'AddColony', 'PercentRenovated', 'PercentLostByDisease']

# Creating a DataFrame from the fetched data, with the specified column names
bee_df = pd.DataFrame(rows_bee, columns=columns_bee)

# Displaying the first few rows of the DataFrame to verify its contents
print(bee_df.head())

In [ ]:
# SQL query to select all columns from the Monitor table
query_monitor = """
    SELECT *
    FROM Monitor
"""
# Execute the SQL query to retrieve data from the Monitor table
cursor.execute(query_monitor)

# Fetch all rows from the executed SQL query
rows_monitor = cursor.fetchall()

# Column headers as they appear in the Monitor table of the database
columns_monitor = ['CentroidLongitude', 'CentroidLatitude', 'StationYear', 
                   'RiskFactorsReportedYear', 'RiskFactorsReportedState']

# Creating a DataFrame from the fetched data, with the specified column names
monitor_df = pd.DataFrame(rows_monitor, columns=columns_monitor)

# Displaying the first few rows of the DataFrame to verify its contents
print(monitor_df.head())

In [ ]:
# SQL query to select all columns from the MonitorStation table
query_monitor_station = """
    SELECT *
    FROM MonitorStation
"""
# Execute the SQL query to retrieve data from the MonitorStation table
cursor.execute(query_monitor_station)

# Fetch all rows from the executed SQL query
rows_monitor_station = cursor.fetchall()

# Column headers as they appear in the MonitorStation table of the database
columns_monitor_station = ['CentroidLongitude', 'CentroidLatitude', 'Year', 'AverageTemperature']

# Creating a DataFrame from the fetched data, with the specified column names
monitor_station_df = pd.DataFrame(rows_monitor_station, columns=columns_monitor_station)

# Displaying the first few rows of the DataFrame to verify its contents
print(monitor_station_df.head())

In [ ]:
# SQL query to select all columns from the Pesticide table
query_pesticide = """
    SELECT *
    FROM Pesticide
"""
# Execute the SQL query to retrieve data from the Pesticide table
cursor.execute(query_pesticide)

# Fetch all rows from the executed SQL query
rows_pesticide = cursor.fetchall()

# Column headers as they appear in the Pesticide table of the database
columns_pesticide = ['Year', 'State', 'LowEstimate', 'HighEstimate']

# Creating a DataFrame from the fetched data, with the specified column names
pesticide_df = pd.DataFrame(rows_pesticide, columns=columns_pesticide)

# Displaying the first few rows of the DataFrame to verify its contents
print(pesticide_df.head())

In [ ]:
# Merge the DataFrames on 'CentroidLongitude', 'CentroidLatitude', and 'Year'
combined_df = pd.merge(
    monitor_station_df, 
    monitor_df, 
    how='inner', 
    left_on=['CentroidLongitude', 'CentroidLatitude', 'Year'],
    right_on=['CentroidLongitude', 'CentroidLatitude', 'StationYear']
)

print(combined_df)

In [ ]:
# Selecting specific columns and creating a new DataFrame
temp_df = combined_df[['RiskFactorsReportedState', 'Year', 'AverageTemperature']].copy()

# Renaming the column 'RiskFactorsReportedState' to 'State'
temp_df.rename(columns={'RiskFactorsReportedState': 'State'}, inplace=True)

# Displaying the new DataFrame
print(temp_df)

In [ ]:
# Merge temp_df with gas_conditions_df on 'State' and 'Year' columns
merged1 = pd.merge(temp_df, gas_conditions_df, how='inner', 
                left_on=['State', 'Year'], right_on=['State', 'Year'])

# select response variable %loss from bee_df, and merge by 'State' and 'Year'
bee1 = bee_df[['PercentLost', 'State', 'Year', 'PercentLostByDisease']].copy()
data1 = pd.merge(merged1, bee1, how='inner', 
                left_on=['State', 'Year'], right_on=['State', 'Year'])

# Pivot the table to transform GasName values into separate columns, with MeanValue as values
data_pivot = data1.pivot_table(index=['State', 'Year', 'AverageTemperature', 
                                      'PercentLost', 'PercentLostByDisease'],
                              columns='GasName', 
                              values='MeanValue').reset_index()

# Rename columns to add '_conc' suffix to gas concentration columns
data_pivot.columns = [str(col) + '_conc' if col not in ['State', 'Year',
                                                        'AverageTemperature', 'PercentLost', 
                                                        'PercentLostByDisease'] else col for col in data_pivot.columns]








# Now merge the pivoted data back with AverageAQI and PercentLost, and also add pesticide data
data2 = pd.merge(data_pivot, data1[['State', 'Year', 'PercentLost', 'PercentLostByDisease']].drop_duplicates(), 
                      on=['State', 'Year', 'PercentLost', 'PercentLostByDisease'], 
                      how='left')
pesticide_df['PesticideEstimate'] = pesticide_df[['LowEstimate', 'HighEstimate']].mean(axis=1)

# Drop the LowEstimate and HighEstimate columns
pesticide_df.drop(['LowEstimate', 'HighEstimate'], axis=1, inplace=True)

data = pd.merge(data2, pesticide_df, on=['State', 'Year'])

# Drop the O3_conc column
data.drop('O3_conc', axis=1, inplace=True)

# Display the final merged DataFrame for analysis
data

Here We excluded AverageAQI from the regression analysis to avoid redundancy and potential multicollinearity, as it is a composite measure of multiple air pollutants already included as individual variables. This approach ensures a clearer interpretation of each pollutant's impact on bee colony loss, facilitating more precise policy recommendations.

Using the percentage lost of bee colonies as the response variable provides a normalized measure that allows for equitable comparison across different states and years, regardless of the total number of colonies. This choice avoids bias associated with varying colony sizes and better reflects the proportional impact of various factors on bee populations.

In [ ]:
# Variables to check for linearity
variables = ['AverageTemperature', 'CO_conc', 'NO2_conc', 'SO2_conc', 'PercentLostByDisease', 'PesticideEstimate']

# Plotting the relationships
for var in variables:
    sns.lmplot(x=var, y='PercentLost', data=data, aspect=1.5)
    plt.title(f'Relationship between {var} and PercentLost')
    plt.show()

In the case of the AverageTemperature versus PercentLost plot, the data points appear to be randomly distributed around the best-fit line, with no discernible pattern. This lack of clear trend or curvature implies that the linearity assumption may hold for this relationship.

For the CO_conc versus PercentLost plot, we see a clear positive trend as CO_conc increases. The data points closely align with the best-fit line, indicating a possible linear relationship.

The scatter plot for NO2_conc and PercentLost also displays a positive trend with data points broadly scattered around the best-fit line. The wide spread of points suggests potential variability in the relationship, but it does not heavily deviate from linearity.

The plot for SO2_conc and PercentLostByDisease shows a very slight trend with a concentration of data points at lower concentration values. The relationship here is less clear and might not be linear, or it may suggest that SO2_conc has a weaker linear relationship with PercentLost compared to other pollutants.

Now let's checking multicollinearity measured by VIF score, where a high value implies the possible existance of multicollinearity. A rule of thumb is that if VIF is greater than 10, it indicates high multicollinearity between this independent variable and the others.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Select features for VIF computation
features = data[['AverageTemperature', 'CO_conc', 'NO2_conc', 'SO2_conc', 'PercentLostByDisease', 'PesticideEstimate']]

# Calculating VIF for each feature
vif_data = pd.DataFrame()
vif_data['Variable'] = features.columns
vif_data['VIF'] = [variance_inflation_factor(features.values, i) for i in range(len(features.columns))]

print(vif_data)

From the table above, since all VIF values are smaller than 10, this indicates we don't need to worry about the problem of multicollinearity in our explanatory variables. However, there is some multicollinear relationship between them, as the VIF score of AverageTemperature and NO2_conc are a bit higher than 5. Based on common sense, it is valid as a higher amount of greenhouse gases released causes temperature to go up. Furthermore, to deduce whether an interaction term between explanatory variables should be included in the regression model, we want to look into the correlation matrix:

In [ ]:
correlation_matrix = data[['AverageTemperature', 'CO_conc', 'NO2_conc', 'SO2_conc', 'PercentLostByDisease', 'PesticideEstimate']].corr()

# Set up the matplotlib figure
plt.figure(figsize=(8, 6))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='Blues', 
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

plt.show()

From the plot, we can see that there is no strong correlation between explanatory variables, so we do not need to include the interaction term in our linear regression model. Then, we want to use ordinal encoding to transform 'State' into numerical values for model fitting, where each unique value represents a state in the US.

##  Linear Regression Model - Ordinary Least Squares (OLS)

In [ ]:
# Initialize the encoder
encoder = OrdinalEncoder()

# Fit and transform the 'State' column
data['State_encoded'] = encoder.fit_transform(data[['State']])

In [ ]:
# Drop the non-numeric 'State' column if it's still in the DataFrame
X = data.drop(['PercentLost', 'State'], axis=1)

# Ensure that 'State_encoded' is used as a predictor
X['State_encoded'] = data['State_encoded']

# Add a constant to the model (intercept)
X = sm.add_constant(X)

y = data['PercentLost']  # Response variable

# Fit the regression model
model = sm.OLS(y, X).fit()

# Print the summary of the regression
print(model.summary())

The linear regression results show that the model explains approximately 11% of the variability in the percent loss of bee colonies, as indicated by the adjusted R-squared value. Among the variables, carbon monoxide concentration (CO_conc) and nitrogen dioxide concentration (NO2_conc) are statistically significant at the 5% level, suggesting a potential association with the percent loss of bee colonies. The significance of these pollutants highlights them as factors that could influence bee colony losses. However, the low R-squared value suggests that the model does not fit the data very well, meaning that there are other unaccounted factors or complex relationships that influence the percent loss of bee colonies. The variables for year, average temperature, ozone concentration (O3_conc), sulfur dioxide concentration (SO2_conc), and state encoded as numeric values were not found to be significant predictors in this model.

In addition to the other unaccounted factors or complex relationships that influence the percent loss of bee colonies, the poor performance of the linear regression model is probably because the association between our explanatory variables and response variables is non-linear. In this case, we want to investigate by fitting a non-linear regression model.

## Non-Linear Regression Model - XGBoost

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Initialize XGBoost regressor
xgb_model = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                             max_depth = 5, alpha = 10, n_estimators = 100)

# Fit the model
xgb_model.fit(X_train, y_train)

# Predict the model
y_pred = xgb_model.predict(X_test)

# Compute and print the performance metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

# Compute SHAP values
explainer = shap.TreeExplainer(xgb_model)
shap_values = explainer.shap_values(X_train)

# Plot the SHAP values - summary plot
shap.summary_plot(shap_values, X_train)

With an R-squared of approximately 0.0667, the XGBoost model has limited predictive power and explains only about 6.67% of the variance in the percent loss of bee colonies. This low R-squared value indicates that the model is not capturing the majority of the variability in the response variable. Even though we haven't tried hyperparameter optimization for the model, the amount of improvement that could be achieved via it is limited. 

The SHAP plot indicates the impact of each feature value on the XGBoost model output for predicting the percent loss of bee colonies. Each point on the plot represents a specific observation from the dataset. The positioning of the points on the x-axis shows the SHAP value, which indicates the impact on the model prediction. Points to the right of the center line suggest a higher percentage loss of bee colonies, while points to the left suggest a lower percentage loss. Overall, the overlap of positive and negative SHAP values across features suggests intricate relationships and potential interactions between variables that the model may not fully capture.

Overall, The low R-squared, alongside a significant MSE, implies that the model's predictions are generally far off from the actual values.s.

In [ ]:
shap.summary_plot(shap_values, X, plot_type="bar")

The plot provides a visual summary of the average impact of each feature on the model output, as estimated by SHAP values. However, given the poor performance of the model, these values should be interpreted with caution.


Based on the SHAP global feature importance plot you’ve shared, it seems that Statd and Year have the most significant impact on the model's output, with high SHAP values indicating a strong influence on the percentage of bee colony loss


Comparing the SHAP results with the coefficients from the linear regression output, we see some common findings. CO_conc and NO2_conc exhibit statistical significance in the linear regression and are also highlighted by SHAP values, though SHAP gives more weight to State_encoded and Year, which were not statistically significant in the linear regressi This may suggest that there is a relatively significant nonlinear relationship between state, year, and the percentage of bee colony losses.on.

By performance of both linear and non-linear models, we can deduce that the previous poor performance of the linear regression model is not because the relationship is non-linear, but because there are actually unaccounted factors that influence the percent loss of bee colonies.

In [ ]:
# Close the cursor and connection
cursor.close()
connection.close()

## Major findings:


## Limitations:

- Temperature Variability: Analyzing monthly temperature data would provide a more comprehensive understanding of temperature fluctuations throughout the year.

- Seasonal Colony Trends: Colony numbers may vary by quarter, with some states exhibiting more pronounced seasonal patterns. Factors such as summer heat and natural bee biological cycles could contribute to these fluctuations.

- Parasite Trends: Parasite infestations, particularly in wintering states, may follow cyclical patterns, impacting bee colony health inconsistently over time.

- State Disparities in Colony Numbers: Variations in total colony numbers between states are influenced by factors such as bee species diversity, habitat preferences, and unique environmental conditions.

- Methodological Considerations: Refining detection methods for colony numbers and pesticide usage is necessary, especially considering the inherent biases introduced by larger states having more colonies and higher pesticide usage.

## Future Directions:

- Fine-Grained Temperature Analysis: Conducting in-depth analysis using monthly temperature data will offer insights into temperature fluctuations and their effects on bee colonies.

- Seasonal Variations Research: Investigating seasonal trends in colony numbers across states can illuminate the relationship between environmental factors and bee biology, informing targeted management strategies.

- Long-Term Parasite Monitoring: Implementing long-term monitoring programs to track parasite trends in wintering states will aid in identifying recurring patterns and developing proactive measures to protect bee health.

- State-Specific Factors Investigation: Further research into factors contributing to differences in colony numbers between states, such as bee species composition and habitat characteristics, will enhance understanding of regional variations in beekeeping practices and pesticide usage.

- Refinement of Methodologies: Developing more sophisticated methodologies to adjust for geographical differences in colony numbers and pesticide usage will improve the accuracy and reliability of data analysis. For example, measuring bee populations or pesticide usage per square foot of area could provide more meaningful insights.